In [ ]:
import ds_loader as dsl
import numpy
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision.utils import make_grid
from torchvision.models.mobilenet import mobilenet_v2
from pathlib import Path


In [ ]:
class Config():
    batch_size = 32
    epochs = 20
    learn_rate = 0.001
    CLASSES = 55
    DATABASE_FOLDER_TRAIN = str(Path("../../dataset/training/"))
    DATABASE_FOLDER_TEST = str(Path("../../dataset/testing/"))
    SAMPLING_RATE = 800
    #weight_decay = 0.001

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
train_dl = dsl.get_dataloader(is_train=True, batch_size=Config.batch_size)
test_dl = dsl.get_dataloader(is_train=False, batch_size=Config.batch_size)

In [ ]:
#define mode
model = mobilenet_v2(pretrained=True)

model.features[0][0] = nn.Conv2d(9, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
model.classifier[1] = nn.Linear(in_features=model.classifier[1].in_features, out_features=Config.CLASSES)

model.to(device)

# define optimizer
#optimizer = torch.optim.Adam(model.parameters(), lr=Config.learn_rate, weight_decay=Config.weight_decay)
optimizer = torch.optim.Adam(model.parameters(), lr=Config.learn_rate)
# define loss function
loss_function = nn.CrossEntropyLoss()

next(model.parameters()).is_cuda

In [ ]:
loss_ls_train = []
model.train()

for epoch in range(Config.epochs):
    for data, labels in train_dl:
        data, labels = data.to(device), labels.to(device)

        optimizer.zero_grad()
        data = data.permute(0, 3, 1, 2)

        prediction = model(data)

        loss = loss_function(prediction, labels)
        loss.backward()
        optimizer.step()

        loss_ls_train.append(loss.data.item())

    print('===> Epoch: {} loss: {:.4f}'.format(epoch, loss.data.item()))

plt.plot(loss_ls_train)
plt.show()

In [ ]:
# Start Evaluation
model.eval()
correct_pred = 0
num_pred = 0

for data, label in test_dl:
    data, label = data.to(device), label.to(device)

    data = data.permute(0, 3, 1, 2)
    pred = torch.argmax(model(data), dim=1)

    num_pred += len(label)
    correct_pred = correct_pred + torch.eq(pred,label).sum().data.item()

print('Number of correct predictions: ' + str(correct_pred))
print('Number of Predictions:' + str(num_pred))
print('Portion: ' + str(correct_pred/num_pred))